In [1]:
import geopandas as gpd
import os
import zipfile as zf
import geopandas
from shapely.geometry import shape
import gzip 
import urllib.request
import xarray as xr
import io
import shapefile
import pandas as pd
import json

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean

In [69]:
df = pd.read_pickle("final.pkl")

In [11]:
df

,time,timeMin,timeMax,gdp
0,1980-01-15,-1.59,1.77,6837.641000
1,1980-02-15,-2.59,6.66,6790.678333
2,1980-03-15,-1.40,3.26,6743.715667
3,1980-04-15,-1.39,3.05,6696.753000
4,1980-05-15,-1.49,2.39,6694.100000
...,...,...,...,...
491,2020-12-15,-0.27,5.58,18989.854000
492,2021-01-15,0.48,7.10,19087.568000
493,2021-02-15,-4.14,1.27,NaN
494,2021-03-15,-1.28,5.49,NaN


In [70]:
df = df.reset_index()
df['time'] = pd.to_datetime(df['time'])
df = df.set_index(df['time'])
df = df.sort_index()
df =  df.drop(["time","index","gdp"], axis =1)

In [71]:
df

,timeMin,timeMax
time,,
1980-01-15,-1.59,1.77
1980-02-15,-2.59,6.66
1980-03-15,-1.40,3.26
1980-04-15,-1.39,3.05
1980-05-15,-1.49,2.39
...,...,...
2020-12-15,-0.27,5.58
2021-01-15,0.48,7.10
2021-02-15,-4.14,1.27


In [25]:
496*0.75

372.0

Scaling dataset

In [72]:
min_max_scaler = MinMaxScaler()
df[["timeMin", "timeMax"]] = min_max_scaler.fit_transform(df[["timeMin", "timeMax"]])

In [73]:
df

,timeMin,timeMax
time,,
1980-01-15,0.662583,0.124825
1980-02-15,0.568332,0.467742
1980-03-15,0.680490,0.229313
1980-04-15,0.681433,0.214586
1980-05-15,0.672008,0.168303
...,...,...
2020-12-15,0.786993,0.392006
2021-01-15,0.857681,0.498597
2021-02-15,0.422243,0.089762


In [74]:
train,test = df[1:372], df[372:] 

In [75]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [76]:
n_past = 50
n_future = 1
n_features = 2

In [77]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [78]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, 100), (None, 41200       input_3[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 1, 100)       0           lstm_6[0][0]                     
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 1, 100)       80400       repeat_vector_2[0][0]            
                                                                 lstm_6[0][1]          

In [79]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, 50, 100), (N 41200       input_4[0][0]                    
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, 100), (None, 80400       lstm_8[0][0]                     
__________________________________________________________________________________________________
repeat_vector_3 (RepeatVector)  (None, 1, 100)       0           lstm_9[0][0]                     
_______________________________________________________________________________________

In [80]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])

Epoch 1/25
11/11 [==============================] - 1s 98ms/step - loss: 0.0550 - val_loss: 0.0130
Epoch 2/25
11/11 [==============================] - 0s 28ms/step - loss: 0.0112 - val_loss: 0.0105
Epoch 3/25
11/11 [==============================] - 0s 25ms/step - loss: 0.0103 - val_loss: 0.0112
Epoch 4/25
11/11 [==============================] - 0s 27ms/step - loss: 0.0096 - val_loss: 0.0105
Epoch 5/25
11/11 [==============================] - 0s 26ms/step - loss: 0.0098 - val_loss: 0.0102
Epoch 6/25
11/11 [==============================] - 0s 27ms/step - loss: 0.0095 - val_loss: 0.0103
Epoch 7/25
11/11 [==============================] - 0s 26ms/step - loss: 0.0093 - val_loss: 0.0098
Epoch 8/25
11/11 [==============================] - 0s 25ms/step - loss: 0.0104 - val_loss: 0.0096
Epoch 9/25
11/11 [==============================] - 0s 26ms/step - loss: 0.0094 - val_loss: 0.0110
Epoch 10/25
11/11 [==============================] - 0s 26ms/step - loss: 0.0092 - val_loss: 0.0098
Epoch 11/

In [81]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e2d2=model_e2d2.predict(X_test)

In [82]:
pred_e1d1

array([[[0.70154095, 0.2743775 ]],

       [[0.6968091 , 0.26876283]],

       [[0.7036882 , 0.2569926 ]],

       [[0.7103477 , 0.25682935]],

       [[0.7176047 , 0.25358194]],

       [[0.71442527, 0.24424745]],

       [[0.70784885, 0.23522367]],

       [[0.7085497 , 0.23900618]],

       [[0.71580034, 0.2448578 ]],

       [[0.7158285 , 0.24768598]],

       [[0.73756677, 0.26851898]],

       [[0.77143973, 0.29649365]],

       [[0.8001413 , 0.3055137 ]],

       [[0.8249328 , 0.30238512]],

       [[0.826325  , 0.303078  ]],

       [[0.8166557 , 0.2917668 ]],

       [[0.81361365, 0.28168508]],

       [[0.80969054, 0.27880365]],

       [[0.79931015, 0.2716334 ]],

       [[0.78804034, 0.26449043]],

       [[0.7879341 , 0.27014244]],

       [[0.80124503, 0.27882406]],

       [[0.7811891 , 0.2788387 ]],

       [[0.77106255, 0.27620932]],

       [[0.77718467, 0.281689  ]],

       [[0.7530216 , 0.27791268]],

       [[0.756707  , 0.2723412 ]],

       [[0.7452439 , 0.25662

In [83]:
pred_e2d2

array([[[0.683442  , 0.24414946]],

       [[0.6857188 , 0.2390319 ]],

       [[0.69353384, 0.23129682]],

       [[0.6987966 , 0.22925086]],

       [[0.7043388 , 0.22727685]],

       [[0.7043925 , 0.2252102 ]],

       [[0.70122385, 0.22445054]],

       [[0.69939286, 0.22815879]],

       [[0.70118594, 0.23217924]],

       [[0.7003465 , 0.23550804]],

       [[0.7097796 , 0.24281998]],

       [[0.72790134, 0.25099128]],

       [[0.74858654, 0.25233212]],

       [[0.7694125 , 0.2502076 ]],

       [[0.77723503, 0.25181845]],

       [[0.77901256, 0.24963488]],

       [[0.7815234 , 0.24756809]],

       [[0.7809533 , 0.24874698]],

       [[0.7762971 , 0.24905778]],

       [[0.7697781 , 0.24941234]],

       [[0.76686776, 0.25326055]],

       [[0.77140915, 0.25699806]],

       [[0.760885  , 0.2595604 ]],

       [[0.755247  , 0.25859097]],

       [[0.75713015, 0.25866267]],

       [[0.7449464 , 0.2571182 ]],

       [[0.7465641 , 0.25186214]],

       [[0.74089456, 0.24545

In [ ]:
y_new_inverse = scalery.inverse_transform(pred_e2d2)